<a href="https://colab.research.google.com/github/devcon14/colab-connect/blob/main/backtests/Backtest_Turnaround_Tuesday_Daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Turnaround Tuesday

* [quantified trader](https://www.quantifiedstrategies.com/turnaround-tuesday/)
* Contradictory to buy Monday strength, maybe on single stocks [chart art](https://www.tradingview.com/script/8zGpvRaM-Buy-Tuesday-Strategy-by-ChartArt)
* ET TurnaroundMonday
* [->](https://www.tablesgenerator.com/markdown_tables)

| Profit Factor | Win Rate | Return | Drawdown |
|---------------|----------|--------|----------|
| 4.93          | 81       | 163%   | 11%      |


In [63]:
if True:
  !rm -rf colab-connect
  !git clone https://github.com/devcon14/colab-connect.git
  import sys
  if not "colab-connect" in sys.path:
    sys.path.append("colab-connect")

ALPHAVANTAGE_API = "C8RFYYD13O6DPWQ2"
API_GLASSNODE = "b07abeb0-382a-42df-aeed-e72a989f240d"
CC_API_KEY = "4c62b8fbe3270e36eec8a2db57e10460da6b1223b198417f9bc17d1989836751"
QUANDL_API = "Nn-WLPogByiCBZmCJQS6"

Cloning into 'colab-connect'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 137 (delta 71), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (137/137), 275.97 KiB | 3.32 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [64]:
%%capture
try:
  from backtesting import Backtest, Strategy
except Exception as e:
  !pip install backtesting

from backtest_lib_stop_entry import get_bt_results
from features_main import get_features_cnd
from features_main import get_features_kaggle


In [65]:
import pandas_datareader.data as web
import pandas as pd

symbol = "SPY"
# symbol = "QQQ"

if "/" in symbol:
  sec = web.DataReader(symbol, "av-forex-daily", api_key=ALPHAVANTAGE_API)
else:
  sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API)
  # sec = web.DataReader(symbol, "av-daily-adjusted", api_key=ALPHAVANTAGE_API, start="2010-01-01")
  # sec["close"] = sec["adjusted close"]

sec["symbol"] = symbol
sec

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol
2002-12-30,87.790001,88.470001,87.220001,88.110001,59.997698,29968000,0.0,1.0,SPY
2002-12-31,87.989998,88.430000,87.110001,88.230003,60.079412,34036600,0.0,1.0,SPY
2003-01-02,88.849998,91.300003,88.540001,91.070000,62.013282,44516300,0.0,1.0,SPY
2003-01-03,90.910004,91.379997,90.500000,91.349998,62.203944,32222600,0.0,1.0,SPY
2003-01-06,91.239998,93.489998,91.169998,92.959999,63.300260,40984500,0.0,1.0,SPY
...,...,...,...,...,...,...,...,...,...
2022-12-19,383.470000,383.820000,378.280000,380.020000,380.020000,79878103,0.0,1.0,SPY
2022-12-20,379.230000,382.230000,377.850000,380.540000,380.540000,74427240,0.0,1.0,SPY
2022-12-21,383.250000,387.410000,382.690000,386.230000,386.230000,78167369,0.0,1.0,SPY
2022-12-22,383.050000,386.213400,374.770000,380.720000,380.720000,100120855,0.0,1.0,SPY


In [66]:
sec["date"] = pd.to_datetime(sec.index)
sec["ft_dt_dow"] = sec["date"].dt.dayofweek

get_features_cnd(sec)
get_features_kaggle(sec)

sec.columns

Index(['open', 'high', 'low', 'close', 'adjusted close', 'volume',
       'dividend amount', 'split coefficient', 'symbol', 'date', 'ft_dt_dow',
       'ft_cnd_range', 'ft_cnd_nr_4', 'ft_cnd_wr_4', 'ft_cnd_nr_7',
       'ft_cnd_wr_7', 'ft_cnd_high_breakout', 'ft_cnd_low_breakout',
       'ft_cnd_lower_shadow', 'ft_cnd_upper_shadow', 'ft_cnd_closing_high',
       'ft_cnd_closing_low', 'ft_cnd_closing_high_1', 'ft_cnd_closing_low_1',
       'tmp_high_1', 'tmp_low_1', 'ft_cnd_inside_bar', 'ft_cnd_engulfing',
       'ft_cnd_green_soldier', 'ft_cnd_red_soldier', 'ft_ta_win_streak',
       'ft_ta_loss_streak', 'ft_dt_bizday-n', 'ft_dt_bizday+n',
       'ft_dt_dow_wom_code'],
      dtype='object')

In [67]:
# SETUP SIGNALS

sec["ft_cnd_closing_low_1"].fillna(False, inplace=True)
sec["ft_cnd_closing_high_1"].fillna(False, inplace=True)

# TODO could make this method of signal a function for boilerplate
sec["all"] = True
sec["signal_long"] = sec.query("ft_cnd_closing_low and ft_cnd_closing_low_1 and ft_dt_dow==1")["all"]
# sec["signal_short"] = sec.query("ft_cnd_closing_high and ft_cnd_closing_high_1 and ft_dt_dow==1")["all"]
sec["signal_long"].fillna(False, inplace=True)
# sec["signal_short"].fillna(False, inplace=True)
sec["signal_short"] = False

sec["signal_long_exit"] = sec["ft_cnd_closing_high"]
sec["signal_short_exit"] = sec["ft_cnd_closing_low"]

sec

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient,symbol,date,...,ft_ta_win_streak,ft_ta_loss_streak,ft_dt_bizday-n,ft_dt_bizday+n,ft_dt_dow_wom_code,all,signal_long,signal_short,signal_long_exit,signal_short_exit
2002-12-30,87.790001,88.470001,87.220001,88.110001,59.997698,29968000,0.0,1.0,SPY,2002-12-30,...,NaN,NaN,-1,20,0:4,True,False,False,False,False
2002-12-31,87.989998,88.430000,87.110001,88.230003,60.079412,34036600,0.0,1.0,SPY,2002-12-31,...,NaN,NaN,0,21,1:4,True,False,False,False,False
2003-01-02,88.849998,91.300003,88.540001,91.070000,62.013282,44516300,0.0,1.0,SPY,2003-01-02,...,NaN,NaN,-21,1,3:0,True,False,False,True,False
2003-01-03,90.910004,91.379997,90.500000,91.349998,62.203944,32222600,0.0,1.0,SPY,2003-01-03,...,NaN,NaN,-20,2,4:0,True,False,False,True,False
2003-01-06,91.239998,93.489998,91.169998,92.959999,63.300260,40984500,0.0,1.0,SPY,2003-01-06,...,NaN,NaN,-19,3,0:0,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-19,383.470000,383.820000,378.280000,380.020000,380.020000,79878103,0.0,1.0,SPY,2022-12-19,...,0.0,4.0,-10,12,0:2,True,False,False,False,True
2022-12-20,379.230000,382.230000,377.850000,380.540000,380.540000,74427240,0.0,1.0,SPY,2022-12-20,...,1.0,0.0,-9,13,1:2,True,False,False,False,False
2022-12-21,383.250000,387.410000,382.690000,386.230000,386.230000,78167369,0.0,1.0,SPY,2022-12-21,...,2.0,0.0,-8,14,2:2,True,False,False,True,False
2022-12-22,383.050000,386.213400,374.770000,380.720000,380.720000,100120855,0.0,1.0,SPY,2022-12-22,...,0.0,1.0,-7,15,3:3,True,False,False,False,True


In [68]:
from backtest_lib_market_entry import MarketStrategy


class TurnAroundStrategy(MarketStrategy):

  def init(self):
    super().init()

    def fn_dow(): return self.data.Ft_dt_dow
    self.dow = self.I(fn_dow, overlay=False)


NameError: ignored

In [62]:
from backtest_lib_stop_entry import get_bt_results
import backtest_lib_stop_entry
backtest_lib_stop_entry.sec = sec

bt = get_bt_results(sec, TurnAroundStrategy)
# bt = get_bt_results(sec)
bt._results

Start                     2002-12-30 00:00:00
End                       2022-12-23 00:00:00
Duration                   7298 days 00:00:00
Exposure Time [%]                    4.808265
Equity Final [$]                 16327.582998
Equity Peak [$]                  16327.582998
Return [%]                           63.27583
Buy & Hold Return [%]              334.581768
Return (Ann.) [%]                    2.485141
Volatility (Ann.) [%]                5.547596
Sharpe Ratio                         0.447967
Sortino Ratio                        0.822613
Calmar Ratio                         0.214869
Max. Drawdown [%]                  -11.565864
Avg. Drawdown [%]                   -2.350954
Max. Drawdown Duration     1240 days 00:00:00
Avg. Drawdown Duration      152 days 00:00:00
# Trades                                   58
Win Rate [%]                        81.034483
Best Trade [%]                       4.377457
Worst Trade [%]                     -3.200438
Avg. Trade [%]                    